In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

## Options

In [2]:
# parse options
problem = 'pavilion_process.json'
problem_subdir = 'results'

recompute_action_states = False
load_external_movements = False

In [3]:
# generic planning options
id_only = 'A273_M0' # None | 'A273_M0'
free_motion_only = False

In [4]:
# client options
viewer = True
diagnosis = False
view_states = False
watch = False
step_sim = False

disable_env = False
reinit_tool = False

## Parse process from json

In [5]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, get_process_path, save_process

In [6]:
process = parse_process(problem, subdir=problem_subdir)
result_path = get_process_path(problem, subdir='results')
if len(process.movements) == 0:
    cprint('No movements found in process, trigger recompute actions.', 'red')
    recompute_action_states = True
if recompute_action_states:
    cprint('Recomputing Actions and States', 'cyan')
    recompute_action_states(process)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results\pavilion_process.json


In [7]:
# force load external if only planning for the free motions
load_external_movements = load_external_movements or free_motion_only or id_only is not None
if load_external_movements:
    ext_movement_path = os.path.dirname(result_path)
    cprint('Loading external movements from {}'.format(ext_movement_path), 'cyan')
    process.load_external_movements(ext_movement_path)
    if recompute_action_states:
        save_process(process, result_path)
        cprint('Recomputed process saved to %s' % result_path, 'green')

Loading external movements from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210128_RemodelFredPavilion\results


In [12]:
process.movements[0].trajectory.points[10]

JointTrajectoryPoint((21.532, -3.787, -3.522, -0.588, -0.517, -0.001, 0.436, 0.602, 0.138), (2, 2, 2, 0, 0, 0, 0, 0, 0), (0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000), (0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000), (0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000), Duration(10, 0))

In [8]:
# import os, json
# from compas.utilities import DataDecoder

# movement = process.movements[0]
# movement_path = os.path.join(ext_movement_path, movement.filepath)
# print(movement_path)
# with open(movement_path, 'r') as f:
#     parsed_data = json.load(f, cls=DataDecoder).data
# #     dict_data = json.load(f)

In [9]:
# jpt_data = dict_data['value']['trajectory']['value']['points'][0]

## Start client

In [13]:
from copy import deepcopy
import compas

current_state = compas.json_loads(compas.json_dumps(process.initial_state))

In [15]:
conf = process.initial_state['robot'].kinematic_config

In [16]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
client, robot, _ = load_RFL_world(viewer=viewer or diagnosis or view_states or watch or step_sim, verbose=False)
set_initial_state(client, robot, process, disable_env=disable_env, reinit_tool=reinit_tool)

In [16]:
pp.disconnect()

In [20]:
process